In [ ]:
from alhambra_mixes import *

mg = Component("MgCl₂", "1 M")
h2o = Component("H₂0")

add_mg = FixedConcentration(mg, "125 mM")
mg_mix = Mix([add_mg], "10× Mg", fixed_total_volume="1 mL")

print(mg_mix)


: 

In [ ]:
from alhambra_mixes import Component, FixedConcentration, Mix
import pandas as pd
from ipydatagrid import DataGrid

# Create components
mg = Component("MgCl₂", "1 M")
h2o = Component("H₂0")

# Create a Mix
add_mg = FixedConcentration(mg, "125 mM")
mg_buffer = Mix([add_mg], "10× Mg", fixed_total_volume="1 mL")

# Convert the Mix output to a pandas DataFrame
mix_output = str(mg_buffer)
output_lines = mix_output.split("\n")[2:]
headers = output_lines[0].split("|")[1:-1]
headers = [header.strip() for header in headers]

data = []
for line in output_lines[2:]:
    row = line.split("|")[1:-1]
    row = [item.strip() for item in row]
    data.append(row)

df = pd.DataFrame(data, columns=headers)

# Display the DataFrame using ipydatagrid
datagrid = DataGrid(df)
datagrid



: 

In [ ]:
import re
import pandas as pd
from ipydatagrid import DataGrid, TextRenderer
from ipywidgets import VBox
from alhambra_mixes import Component as AlhambraComponent, FixedConcentration

class Component(AlhambraComponent):
    def __init__(self, name, concentration, location=None, well=None):
        super().__init__(name, concentration)
        self._location = location
        self._well = well

    @property
    def location(self):
        return self._location

    @location.setter
    def location(self, value):
        self._location = value

    @property
    def well(self):
        return self._well

    @well.setter
    def well(self, value):
        self._well = value

# Initialize the input DataFrame with a few empty rows
input_data = {
    'Name': ['MgCl₂', 'H₂0', '', '', ''],
    'Concentration': ['1 M', '', '', '', ''],
    'Location': ['', '', '', '', ''],
    'Well': ['', '', '', '', ''],
}
input_df = pd.DataFrame(input_data)

# Initialize the output DataFrame with column names
output_data = {
    'Component': [],
    'Concentration': [],
    'Location': [],
    'Well': [],
    'Mixing': [],
    'Volume': [],
    'Ratio': [],
    'Weight': [],
}
output_df = pd.DataFrame(output_data)

# Create DataGrids for input and output
input_datagrid = DataGrid(input_df, editable=True)
output_datagrid = DataGrid(output_df)

def mix_to_dict(mix):
    component_name = mix.components[0].name
    component_concentration = mix.components[0].concentration
    fixed_concentration = mix.fixed_concentration
    total_volume = mix.tx_volume
    number_of_components = len(mix.components)
    
    return {
        "Component": component_name,
        "Src": component_concentration,
        "Dest": fixed_concentration,
        "#": number_of_components,
        "Total Tx Vol": total_volume
    }


# Event handler for input data changes
def on_change(event=None):
    global output_df
    
    if event is not None:
        # Process the input DataFrame
        input_df = pd.DataFrame(event['data'])
    else:
        input_df = input_datagrid.data
    
    # Clear the output DataFrame
    #output_df = output_df.iloc[0:0]
    
    # Process the input DataFrame
    for idx, row in input_df.iterrows():
        # Create Components and Mixes based on the input data
        # Update the output DataFrame accordingly
        name = row['Name']
        concentration = row['Concentration']
        location = row['Location']
        well = row['Well']
        
        if not name or not concentration:
            continue
        
        component = Component(name, concentration, location, well)
        
        # Create Mixes based on the input data
        # For example, let's assume we want to make a FixedConcentration Mix for each component
        target_concentration = "125 mM"  # You can adjust this value based on your needs
        mix = FixedConcentration(component, target_concentration) 
        
        # Update the output DataFrame
        mix_dict = mix_to_dict(mix)
        mix_dict["Location"] = location
        mix_dict["Well"] = well

        print(mix_dict)
        
        output_df = pd.concat([output_df, pd.DataFrame([mix_dict])], ignore_index=True)

        print(output_df)
    
    # Update the output DataGrid
    output_datagrid.data = output_df

# Attach the on_change event handler to the input_datagrid
input_datagrid.observe(on_change, 'data')  # Use 'data' instead of input_df

# Call the on_change event handler manually to update the output spreadsheet
on_change()

# Display the input and output DataGrids in a VBox
VBox([input_datagrid, output_datagrid])


: 

In [ ]:
import pandas as pd
from ipydatagrid import DataGrid
from ipywidgets import VBox
from alhambra_mixes import Component as AlhambraComponent, FixedConcentration

# class Component(AlhambraComponent):
#     def __init__(self, name, concentration, location=None, well=None):
#         super().__init__(name, concentration)
#         self._location = location
#         self._well = well

#     @property
#     def location(self):
#         return self._location

#     @location.setter
#     def location(self, value):
#         self._location = value

#     @property
#     def well(self):
#         return self._well

#     @well.setter
#     def well(self, value):
#         self._well = value

# Initialize the input DataFrame with a few empty rows
input_data = {
    'Name': ['MgCl₂', 'H₂0', '', '', ''],
    'Concentration': ['1 M', '', '', '', ''],
    'Location': ['', '', '', '', ''],
    'Well': ['', '', '', '', ''],
}
input_df = pd.DataFrame(input_data)

# Create DataGrid for input
input_datagrid = DataGrid(input_df, editable=True)

# Event handler for input data changes
def on_change(event=None):
    if event is not None:
        # Process the input DataFrame
        input_df = pd.DataFrame(event['data'])
    else:
        input_df = input_datagrid.data
    
    # df = pd.DataFrame(data, columns=headers)

    # Display the DataFrame using ipydatagrid
    datagrid = DataGrid(input_df)
    datagrid

    # Process the input DataFrame
    for idx, row in input_df.iterrows():
        name = row['Name']
        concentration = row['Concentration']
        location = row['Location']
        well = row['Well']
        
        if not name or not concentration:
            continue
        
        component = Component(name, concentration, location, well)
        
        # Create Mixes based on the input data
        # For example, let's assume we want to make a FixedConcentration Mix for each component
        target_concentration = "125 mM"  # You can adjust this value based on your needs
        mix = FixedConcentration(component, target_concentration)

# Attach the on_change event handler to the input_datagrid
input_datagrid.observe(on_change, 'data')  # Use 'data' instead of input_df

# Call the on_change event handler manually to update the output spreadsheet
on_change()

# Display the input DataGrid in a VBox
VBox([input_datagrid])


